MIT License

Copyright (c) 2025 Zamsheed Khan

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [41]:
import pprint, requests, subprocess, socket, ssl
from urllib.parse import urlparse

# Motivation
I'm in the process of documentation and developing a template for developing, testing and hosting a ReactJS (Typescript) and ASP.NET Core Web API Web Stack Application hosted on IIS. I started using [Bruno](https://www.usebruno.com) for API documentation and testing. But I did not want to disable the SSL verification check. And it looks like I can't because NodeJs (Bruno is built with Electron) doesn't trust system's certificate store by default on Windows. And neither does Postman. 

Maybe on Linux, MacOS, Bruno and Postman would work. But on Windows looks like it doesn't. So I had a few choices in terms of testing SSL, I could have:  
* Written a C# library in .NET Core which utilise the `HttpClient` and `HttpClientHandler` classes to do the SSL/TLS inspection or,
* .NET Framework do the same thing and use `HttpClient` and also `ServicePointManager`. 
* And then write equivalent Windows PowerShell or PowerShell Core to do my initial checking and the incorporate these custom libraries with my Unit testing with MS Unit Test. 

But going with Python instead (because mostly been writing PowerShell Script, need to do something different 😊)

# Introduction
The main reason why I chose python is I want similar behavior to Bruno and Python has a libraries for SSL testing and validation and also for doing API calls, along with the `unittest` module for unit testing. I am going to give it a try with this one then.

In [42]:
url = 'https://localhost:7276/weatherforecast'

If you do an API with PowerShells `Invoke-RestMethod` or `Invoke-WebRequest`, you'll get no certificate warning or errors, because your certificate is already trusted because it's in the certificate store and valid.

# SSL Error
To show you what I mean, I'm going to do an API call, which will throw a certificate error.

In [43]:
requests.get(url)

SSLError: HTTPSConnectionPool(host='localhost', port=7276): Max retries exceeded with url: /weatherforecast (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1077)')))

As you can see I got myself an SSL Error and it couldn't verify the certificate. And just to be sure, Python also has the ability to disable the SSL Verification Check.

In [44]:
requests.get(url, verify=False)

C:\Users\zamk1\AppData\Roaming\Python\Python314\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

Although I get a HTTP 200 response, I also get a warning about making requests without certificate verification. 

# Exporting Self Signed Cert
My self signed certificate is sitting in my Personal - Current User Certificate Store in Windows. You can export the self signed cert with the `dotnet` with the command `dotnet dev-certs https --format pem -ep Z:\Projects\random-scripts-repo\Python\SSLTesting\Cert\localhost.crt`. I'm using the `subprocess.run` to do it for me.

In [45]:
CertName = 'localhost-ca.crt'
CertLocation = 'Z:\\Projects\\random-scripts-repo\\Python\\SSLTesting\\Cert\\'
FullCertPath = f'{CertLocation}{CertName}'
RelativeCertPath = f'.\\Cert\\{CertName}'

In [47]:
results = subprocess.run(['dotnet', 'dev-certs', 'https', '--format', 'pem', '-ep', FullCertPath],
            stdout=subprocess.PIPE,
            stderr= subprocess.PIPE,
            text=True,
            check=False
        )

strResults = ""

if results.returncode != 0:
    strResults = f"dotnet Error: {results.stderr}\n"

strResults = strResults + f"dotnet Output: {results.stdout}"

print(strResults)

dotnet Output: A valid HTTPS certificate is already present.



Since I am running this on Windows, I'm not really fussed about importing the `os` module and and using the `os.sep` to define the separator for different platforms. `\\` is good enough for me 😊.

Now I can do my request with my exported certificate.

In [48]:
response = requests.get(url, verify=RelativeCertPath)

pprint.pp(f"""
Response Code:     {response.status_code}
Respond Headers:   {response.headers}
Response Body:     {response.text}
""")

('\n'
 'Response Code:     200\n'
 "Respond Headers:   {'Content-Type': 'application/json; charset=utf-8', "
 "'Date': 'Sat, 27 Dec 2025 15:24:25 GMT', 'Server': 'Kestrel', "
 "'Transfer-Encoding': 'chunked'}\n"
 'Response Body:     '
 '[{"date":"2025-12-29","temperatureC":24,"temperatureF":75,"summary":"Freezing"},{"date":"2025-12-30","temperatureC":50,"temperatureF":121,"summary":"Warm"},{"date":"2025-12-31","temperatureC":53,"temperatureF":127,"summary":"Scorching"},{"date":"2026-01-01","temperatureC":-15,"temperatureF":6,"summary":"Mild"},{"date":"2026-01-02","temperatureC":25,"temperatureF":76,"summary":"Sweltering"}]\n')


No warnings or errors, which I what I want to see.

# Certificate Validation
I got a bit curious to see how can I validate my certificate, you can use `socket` to create a connection and then wrap the connection with the context which you create with the `ssl` library. 

In [ ]:
parsedUrl = urlparse(url)
hostname = parsedUrl.hostname
port = parsedUrl.port or 443

try:
    context = ssl.create_default_context()
    context.load_verify_locations(RelativeCertPath)

    with socket.create_connection((hostname, port)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as ssock:
            pprint.pp(f"SSL established. Peer: {ssock.getpeercert()}")
except Exception as e:
    print(f"An error occurred: {e}")

The last cell, I cleared it out. But you can definitely try it! So going forward for API testing, I'm sticking to Python and so for SSL verification and testing as well. 

# References:  
* [Unable to verify the first certificate, self-signed certificate not working? · Issue #4949 · usebruno/bruno](https://github.com/usebruno/bruno/issues/4949)
* [ssl — TLS/SSL wrapper for socket objects &#8212; Python 3.14.2 documentation](https://docs.python.org/3/library/ssl.html)